In [ ]:
library(ergm)
library(statnet)
library(texreg)
library(readxl)
library(igraph)

In [ ]:
data_matrix <- read.csv("adjacency_matrix.csv", header = TRUE, row.names = 1)
set.seed(122)
matrix <- as.matrix(data_matrix)
d <- graph_from_adjacency_matrix(matrix, mode = c('directed'),weighted = 'translation_based',diag = TRUE)
plot(d,vertex.size = 3, layout = layout.fruchterman.reingold, vertex.color = "red", vertex.label = V(d)$name, edge.arrow.size = 0.5, edge.width = E(d)$translation_based*0.1,
    vertex.label.degree = pi/4, vertex.label.dist = 1,
    vertex.label.color = "black",edge.color = "black")
matrix <- as.matrix(data_matrix)
net <- as.network(matrix, matrix.type = "adjacency", directed = TRUE)
dgraph <- graph_from_adjacency_matrix(matrix, mode = c('directed'),weighted = 'translation_based',diag = TRUE)

In [ ]:
#small-world
clust.coef.dir <- function(graph){
    A <- as.matrix(get.adjacency(graph))
    S <- A + t(A)
    deg <- degree(graph,mode=c("total"))
    num <- diag(S %*% S %*% S)
    denom <- diag(A %*% A)
    denom <- 2 * (deg * (deg - 1)- 2 * denom)
    cl <- mean(num / denom)
    return(cl)}
clust.coef.dir <- function(graph){
    A <- as.matrix(get.adjacency(graph))
    S <- A + t(A)
    deg <- degree(graph, mode=c("total"))
    num <- diag(S %*% S %*% S)
    denom <- diag(A %*% A)
    denom <- 2 * (deg * (deg - 1) - 2 * denom)
    valid_indices <- denom != 0
    if (any(valid_indices)) {
        cl <- mean(num[valid_indices] / denom[valid_indices])
    } else {
        cl <- 0
    }
        return(cl)}

ntrials <- 1000
nv <- vcount(dgraph)
ne <- ecount(dgraph)
c1.rg <- numeric(ntrials)
apl.rg <-numeric(ntrials)
for (i in (1:ntrials)){
    g.rg <- erdos.renyi.game(nv,ne,type = "gnm",directed=TRUE)
    c1.rg[i] <- clust.coef.dir(g.rg)
    apl.rg[i] <- mean_distance(g.rg)}

summary(c1.rg)
summary(apl.rg)
clust.coef.dir(dgraph)
mean_distance(dgraph)

results <- data.frame(
  c1_rg = c1.rg,
  apl_rg = apl.rg)

In [ ]:
#degree
in_degrees <- degree(d, mode = "in")
out_degrees <- degree(d, mode = "out")
degrees <- degree(d)
degree_data <- data.frame(node = names(in_degrees), degree = degrees, in_degree = in_degrees, out_degree = out_degrees)
sorted_degree_data <- degree_data[order(degree_data$degree, decreasing = TRUE), ]